#### The requests package is installed

In [97]:
!pip install requests

#### The BeautifulSoup and Requests library are imported

In [98]:
from bs4 import BeautifulSoup
import requests

In [136]:
result=requests.get('https://www.thomann.de/gb/dreadnought_guitars.html')

In [137]:
result.status_code

200

In [100]:
result.headers

{'Date': 'Mon, 05 Dec 2022 22:30:40 GMT', 'Content-Security-Policy': "frame-ancestors 'self' app.storyblok.com", 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=utf-8', 'Connection': 'close', 'Server': 'Apache'}

In [139]:
src=result.text

In [140]:
soup= BeautifulSoup(src,'lxml')

In [141]:
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html class="no-js" lang="en">
 <head>
  <link crossorigin="" href="https://fonts.static-thomann.de" rel="preconnect"/>
  <link href="https://fonts.static-thomann.de/pics/fonts/open-sans.css?v=1" rel="stylesheet"/>
  <meta content="utf-8" name="charset"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   Dreadnought Acoustic Guitars – Thomann UK
  </title>
  <meta content="Dreadnought Acoustic Guitars at Europe's largest retailer of musical instruments - fast delivery, 30-Day Money-Back Guarantee and 3-year Thomann warranty" name="description"/>
  <meta content="en-GB" name="Language"/>
  <meta content="en-GB" name="Content-Language"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <script type="text/javascript">
   if ((window.navigator.platform.match(/iPhone|iPod/) !

#### An object containing the 

In [146]:
dread_guitars=soup.find('div',class_ = "js-search")

In [157]:
print(dread_guitars.prettify())

<div class="js-search">
 <div class="search-header">
  <div class="headline-row">
   <div class="header-headline">
    <h1 class="header-headline__label fx-headline fx-headline--1 fx-space-bottom--xs">
     Dreadnought Acoustic Guitars
    </h1>
   </div>
  </div>
  <div class="header-actions__container">
  </div>
 </div>
 <div class="fx-grid">
  <div class="fx-grid__col fx-col--12 fx-col--lg-3">
  </div>
  <div class="fx-grid__col fx-col--12 fx-col--lg-9">
   <div>
    <div class="fx-product-list-entry">
     <div class="product">
      <a class="product__image" href="fender_cd_60s_black_wn.htm?listPosition=0&amp;type=category">
       <picture class="fx-picture">
        <source data-srcset="https://thumbs.static-thomann.de/thumb//thumb220x220/pics/prod/455910.webp" srcset="" type="image/webp">
         <source data-srcset="https://thumbs.static-thomann.de/thumb//thumb220x220/pics/prod/455910.jpg" srcset="">
          <img alt="Fender CD-60S Black WN" class="fx-image responsive-image

In [221]:
collection=dread_guitars.find_all('div', class_="fx-product-list-entry")
type(collection)

bs4.element.ResultSet

In [228]:
collection=dread_guitars.find_all('div', class_="fx-product-list-entry")

Manufacturer_list=[]
Model_list=[]
Price_list=[]
Product_availability_list=[]


for guitar in collection:

            Product=guitar.find('a', class_='product__content')

            Manufacturer=Product.find('span', class_='title__manufacturer').text
            Manufacturer_list.append(Manufacturer)

            Model=Product.find('span', class_='title__name').text
            Model_list.append(Model)
            
            Price_unclean=Product.find('div', class_= 'product__price').text

            Price_unclean.strip()

            Price=Price_unclean.strip().lstrip('£')

            Price
            Price_list.append(Price)

            try:
                Product_availability_unclean=Product.find('div',class_='product__availability fx-availability fx-availability--in-stock').text

                Product_availability_unclean

                Product_availability=Product_availability_unclean.strip()

                Product_availability
                Product_availability_list.append(Product_availability)
            
            except:
                Product_availability= None
                Product_availability_list.append(Product_availability)

In [227]:
Price_unclean

'\n\n            £3,299\n    \n\n'